In [3]:
# backend/main.py - FastAPI Backend for William Tanna Website Chatbot

import os
import re
from pathlib import Path
from dotenv import load_dotenv
from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from pydantic import BaseModel
from openai import OpenAI

# ---------------------------
# Config
# ---------------------------

BASE_DIR = Path.cwd().resolve().parent
ENV_PATH = BASE_DIR / ".env"
KB_PATH = BASE_DIR / "knowledge_base.txt"

load_dotenv(dotenv_path=ENV_PATH)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
if not OPENAI_API_KEY:
    raise ValueError("OPENAI_API_KEY not found. Ensure backend/.env exists and has OPENAI_API_KEY=...")

client = OpenAI(api_key=OPENAI_API_KEY)

app = FastAPI(title="William Tanna Portfolio API", version="1.0")

app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],  # later: set to ["https://yourdomain.com"]
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

# ---------------------------
# Models
# ---------------------------

class ChatRequest(BaseModel):
    message: str

class ChatResponse(BaseModel):
    success: bool
    response: str          # highlighted HTML
    raw_response: str      # plain text
    used_kb: bool

# ---------------------------
# Helpers
# ---------------------------

def load_knowledge_base() -> str:
    if KB_PATH.exists():
        return KB_PATH.read_text(encoding="utf-8").strip()

    return (
        "NAME: William Tanna\n"
        "NOTE: Knowledge base file is not added yet.\n"
        "If asked for details, say you don't have enough information."
    )

def highlight_text(text: str) -> str:
    text = re.sub(r'(\d+(\.\d+)?)%', r'<span class="highlight-stat">\\1%</span>', text)
    text = re.sub(r'(\\$[\\d,]+(\\.\\d+)?\\s*[KMB]?)', r'<span class="highlight-money">\\1</span>', text)
    text = re.sub(r'(₹\\s?[\\d,]+(\\.\\d+)?\\s*Cr)', r'<span class="highlight-money">\\1</span>', text)
    text = re.sub(r'(\\d{2,})\\+', r'<span class="highlight-number">\\1+</span>', text)

    keywords = [
        "improved", "decreased", "increased", "analyzed", "managed", "led",
        "built", "developed", "created", "delivered", "automated", "verified",
        "ensured", "authored", "proposed", "mentored", "coordinated"
    ]
    for kw in keywords:
        text = re.sub(rf"\\b({kw})\\b", r'<strong class="highlight-keyword">\\1</strong>', text, flags=re.IGNORECASE)

    return text

def build_system_prompt(kb: str) -> str:
    return f"""
You are the AI assistant embedded in William Tanna's personal website.

Rules:
- Answer using ONLY the knowledge base below.
- If the answer is not in the knowledge base, reply: "I don’t have that info on the website yet."
- Keep it professional, conversational, and confident.
- Use metrics when available. Don’t invent facts.

KNOWLEDGE BASE:
{kb}
""".strip()

# ---------------------------
# Routes
# ---------------------------

@app.get("/api/health")
def health():
    return {
        "status": "healthy",
        "message": "Backend is running",
        "knowledge_base_loaded": KB_PATH.exists()
    }

@app.post("/api/chat", response_model=ChatResponse)
def chat(req: ChatRequest):
    user_message = (req.message or "").strip()
    if not user_message:
        raise HTTPException(status_code=400, detail="No message provided")

    kb = load_knowledge_base()
    system_prompt = build_system_prompt(kb)

    completion = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {"role": "developer", "content": system_prompt},
            {"role": "user", "content": user_message},
        ],
        temperature=0.55,
        max_tokens=500,
    )

    raw = completion.choices[0].message.content.strip()
    highlighted = highlight_text(raw)

    return ChatResponse(
        success=True,
        response=highlighted,
        raw_response=raw,
        used_kb=bool(kb and "Knowledge base file is not added yet" not in kb)
    )


ValueError: OPENAI_API_KEY not found. Ensure backend/.env exists and has OPENAI_API_KEY=...